In [24]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

# Load the dataset
file_path = '/content/BigBasket Products.csv'
data = pd.read_csv(file_path)

# Display the first few rows of the dataset to understand its structure
data.head()


,index,product,category,sub_category,brand,sale_price,market_price,type,rating,description
0,1,Garlic Oil - Vegetarian Capsule 500 mg,Beauty & Hygiene,Hair Care,Sri Sri Ayurveda,220.0,220.0,Hair Oil & Serum,4.1,This Product contains Garlic Oil that is known...
1,2,Water Bottle - Orange,"Kitchen, Garden & Pets",Storage & Accessories,Mastercook,180.0,180.0,Water & Fridge Bottles,2.3,"Each product is microwave safe (without lid), ..."
2,3,"Brass Angle Deep - Plain, No.2",Cleaning & Household,Pooja Needs,Trm,119.0,250.0,Lamp & Lamp Oil,3.4,"A perfect gift for all occasions, be it your m..."
3,4,Cereal Flip Lid Container/Storage Jar - Assort...,Cleaning & Household,Bins & Bathroom Ware,Nakoda,149.0,176.0,"Laundry, Storage Baskets",3.7,Multipurpose container with an attractive desi...
4,5,Creme Soft Soap - For Hands & Body,Beauty & Hygiene,Bath & Hand Wash,Nivea,162.0,162.0,Bathing Bars & Soaps,4.4,Nivea Creme Soft Soap gives your skin the best...


In [25]:
# Checking for missing values
missing_values = data.isnull().sum()

# Checking for duplicate entries
duplicate_entries = data.duplicated().sum()

# Overview of the rating distribution
rating_distribution = data['rating'].describe()

missing_values, duplicate_entries, rating_distribution


(index              0
 product            1
 category           0
 sub_category       0
 brand              1
 sale_price         1
 market_price       1
 type               1
 rating          5958
 description       81
 dtype: int64,
 0,
 count    12987.000000
 mean         3.938654
 std          0.744717
 min          1.000000
 25%          3.700000
 50%          4.100000
 75%          4.300000
 max          5.000000
 Name: rating, dtype: float64)

In [26]:

# Drop rows with missing 'rating' as they are critical for popularity-based recommendations
data.dropna(subset=['rating'], inplace=True)

categorical_features = ['category','sub_category','brand']
label_encoders = {}

for feature in categorical_features:
    le = LabelEncoder()
    data[feature] = le.fit_transform(data[feature].astype(str))  # Use 'le' instead of 'encoder'
    label_encoders[feature] = le

data['description'] = data['description'].astype(str)

# Now, proceed with tokenizing the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data['description'])
text_sequences = tokenizer.texts_to_sequences(data['description'])
text_padded = pad_sequences(text_sequences, maxlen=100)

# Normalize numerical features
scaler = MinMaxScaler()
data['rating'] = scaler.fit_transform(data[['rating']])

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(text_padded, data['rating'], test_size=0.2, random_state=42)


In [27]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Embedding, Flatten, Dropout

input_dim = text_padded.shape[1]  # This is the maxlen of your text sequences

# Define the encoder
input_layer = Input(shape=(input_dim,))
encoded = Dense(128, activation='relu')(input_layer)
encoded = Dense(64, activation='relu')(encoded)

# Define the decoder
decoded = Dense(64, activation='relu')(encoded)
decoded = Dense(128, activation='relu')(decoded)
output_layer = Dense(input_dim, activation='sigmoid')(decoded)

# Compile the autoencoder
autoencoder = Model(input_layer, output_layer)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

# Summary of the model
autoencoder.summary()


Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 100)]             0         
                                                                 
 dense_10 (Dense)            (None, 128)               12928     
                                                                 
 dense_11 (Dense)            (None, 64)                8256      
                                                                 
 dense_12 (Dense)            (None, 64)                4160      
                                                                 
 dense_13 (Dense)            (None, 128)               8320      
                                                                 
 dense_14 (Dense)            (None, 100)               12900     
                                                                 
Total params: 46564 (181.89 KB)
Trainable params: 46564 (18

In [28]:
autoencoder.fit(X_train, X_train, epochs=50, batch_size=256, validation_split=0.2)


Epoch 1/50
33/33 [==============================] - 2s 9ms/step - loss: -16822826.0000 - val_loss: -113450912.0000
Epoch 2/50
33/33 [==============================] - 0s 5ms/step - loss: -2073578752.0000 - val_loss: -9366234112.0000
Epoch 3/50
33/33 [==============================] - 0s 5ms/step - loss: -57728086016.0000 - val_loss: -174877147136.0000
Epoch 4/50
33/33 [==============================] - 0s 5ms/step - loss: -571909144576.0000 - val_loss: -1358683963392.0000
Epoch 5/50
33/33 [==============================] - 0s 5ms/step - loss: -3192796217344.0000 - val_loss: -6366860673024.0000
Epoch 6/50
33/33 [==============================] - 0s 5ms/step - loss: -12268139970560.0000 - val_loss: -21696371228672.0000
Epoch 7/50
33/33 [==============================] - 0s 5ms/step - loss: -36713988620288.0000 - val_loss: -59129670402048.0000
Epoch 8/50
33/33 [==============================] - 0s 5ms/step - loss: -91393598947328.0000 - val_loss: -137519333113856.0000
Epoch 9/50
33/33 [==

In [29]:
# Extract the encoder part of the autoencoder
encoder_model = Model(inputs=autoencoder.input, outputs=autoencoder.layers[-3].output)  # Assuming -3 is the encoded layer

# Generate embeddings for the entire dataset
item_embeddings = encoder_model.predict(text_padded)


406/406 [==============================] - 1s 2ms/step


In [35]:
def recommend_items(item_id, similarity_matrix, items, label_encoders, top_k=3):
    # Fetch the category of the specified item
    item_category = items.iloc[item_id]['category']

    # Get similarity scores for the item
    similarity_scores = list(enumerate(similarity_matrix[item_id]))

    # Sort the items based on the similarity scores
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)

    # Initialize a list to hold indices of items that match the category constraint
    filtered_indices = []

    # Iterate over sorted similarity scores and filter by category
    for idx, score in similarity_scores:
        if items.iloc[idx]['category'] == item_category and idx != item_id:
            filtered_indices.append(idx)
        if len(filtered_indices) == top_k:
            break

    # Fetch the recommended items
    recommended_items = items.iloc[filtered_indices]

    # Decode the categorical features for readability
    for feature in ['category', 'sub_category', 'brand']:
        recommended_items[feature] = label_encoders[feature].inverse_transform(recommended_items[feature])

    # Sort the recommended items in descending order of rating
    recommended_items_sorted = recommended_items.sort_values(by='rating', ascending=False)

    return recommended_items_sorted


In [36]:
# Updated example usage
recommended_items = recommend_items(6, similarity_matrix, data, label_encoders, top_k=3)
print(recommended_items[['product', 'category', 'sub_category', 'brand', 'rating']])


                                        product          category  \
259  Aeda Glycerine Bathing Bar - Natural Green  Beauty & Hygiene   
186             Herbal Wine Grapefruit Lip Balm  Beauty & Hygiene   
354                 Active Fresh Gel Toothpaste  Beauty & Hygiene   

         sub_category             brand  rating  
259  Bath & Hand Wash  K.P. Namboodiris   0.925  
186         Skin Care     Khadi Natural   0.850  
354         Oral Care          Himalaya   0.825  


<ipython-input-35-dba5e3d2ea98>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recommended_items[feature] = label_encoders[feature].inverse_transform(recommended_items[feature])
<ipython-input-35-dba5e3d2ea98>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recommended_items[feature] = label_encoders[feature].inverse_transform(recommended_items[feature])
<ipython-input-35-dba5e3d2ea98>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_ind

In [32]:
# Evaluate the model on the test set
test_loss = autoencoder.evaluate(X_test, X_test)
print(f'Test Loss: {test_loss}')


82/82 [==============================] - 0s 2ms/step - loss: -2179261870504935424.0000
Test Loss: -2.1792618705049354e+18


In [33]:
item_0_data = data.iloc[0]
item_0_data

index                                                           1
product                    Garlic Oil - Vegetarian Capsule 500 mg
category                                                        2
sub_category                                                   42
brand                                                        1489
sale_price                                                  220.0
market_price                                                220.0
type                                             Hair Oil & Serum
rating                                                      0.775
description     This Product contains Garlic Oil that is known...
Name: 0, dtype: object